In [1]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import json
import time
import pickle
import os
import shutil
import skimage as sk
from skimage import io
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from overall import *
import smtplib
import json
import imghdr
from email.message import EmailMessage
from skimage.transform import resize
from sql_functions import *
import mysql.connector
from mysql.connector import errorcode
import pandas as pd

with open('secret.json') as json_file:
    env_variables = json.load(json_file)
    os.environ['EMAIL_ADD'] = env_variables['EMAIL_ADD']
    os.environ['EMAIL_PASS'] = env_variables['EMAIL_PASS']
    os.environ['AZURE'] = env_variables['AZURE']
    os.environ['SQL_HOST'] = env_variables['SQL_HOST']
    os.environ['SQL_USER'] = env_variables['SQL_USER']
    os.environ['SQL_PASS'] = env_variables['SQL_PASS']
    os.environ['SQL_DB'] = env_variables['SQL_DB']
config = {
  'host':os.environ.get('SQL_HOST'),
  'user':os.environ.get('SQL_USER'),
  'password':os.environ.get('SQL_PASS'),
  'database':os.environ.get('SQL_DB')
}

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'azure.storage'; 'azure' is not a package

In [ ]:
!pip uninstall azure -y


In [ ]:
azure_key = os.environ.get('AZURE')
EMAIL_ADD = os.environ.get('EMAIL_ADD')
EMAIL_PASS = os.environ.get('EMAIL_PASS')
blob_service_client = BlobServiceClient.from_connection_string(azure_key)

In [3]:
def im_resize(fname):
    img = io.imread(fname)
    img = resize(img, (800, 600, 3))
    io.imsave(fname, img)

In [4]:
containers = blob_service_client.list_containers() 
cs = {}
for c in containers:
    cs[c.name] = c.last_modified
container_name = sorted(cs, key=cs.get, reverse=True)[0]

In [5]:
def send_email(file_name, obj_name):
    msg = EmailMessage()
    msg['Subject'] = 'Rectify: Item wrongly recycled'
    msg['From'] = EMAIL_ADD
    msg['To'] = 'rectify.hackathon@gmail.com'
    msg.set_content('Oops, you have incorrectly placed an {} into recycling ;)'.format(obj_name))
    
    file = open("status_email.txt","w+") 
    file.write("Rectifying") 
    file.close()

    with open(file_name, 'rb') as f:
        file_data = f.read()
        file_type = imghdr.what(f.name)
        file_name = f.name
    msg.add_attachment(file_data, maintype='image', subtype=file_type, filename=file_name)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:

        smtp.login(EMAIL_ADD, EMAIL_PASS)

        # Args: sender, reciever, message
        smtp.send_message(msg)

In [6]:
def send_reward():
    msg = EmailMessage()
    msg['Subject'] = 'Rectify Reward: You are a star!'
    msg['From'] = EMAIL_ADD
    msg['To'] = 'rectify.hackathon@gmail.com'
    msg.set_content('You have been recycling correctly for 10 days, keep it up!')

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:

        smtp.login(EMAIL_ADD, EMAIL_PASS)

        # Args: sender, reciever, message
        smtp.send_message(msg)

In [7]:
clear_items(1, config)
clear_items(2, config)

set_consecutive_days(1, 9, config)
    
users_to_df(config)

Items Cleared.
Items Cleared.
User 1 consecutive_days => 9
Done.


,userID,name,consecutive_days
0,1,RaspPi,9
1,2,BluePi,0


In [8]:
def check_recycle(objects, fname):
    recycle_dict = ['water_bottle', 'mixing_bowl']
    flag = False
    user = 1
    for obj in objects:
        if obj not in recycle_dict:
            flag = True
        
        add_item(obj, not flag, user, config)
        check_clean(user, 1000, config) 
    reward = False
    if len(objects)!=0:
        reward = check_reward(user, 10, config)
        file = open("status_email.txt","w+") 
        file.write("Reward Sent") 
        file.close()
    if reward == True:
        send_reward()
    if flag:
        print("Non-recyclable object detected! Rectifying...\n\n")
        send_email(fname, objects[0])

In [9]:
import time
import shutil
from datetime import datetime
image_history = []
obj_list_history = []
SSIM_THRESH = 0.5
pickle.dump([], open( "item_list.pkl", "wb" ) )

while True:
    # get latest date
    container_client = blob_service_client.get_container_client(container_name)
    container_client.list_blobs()
    blobs = []
    for i in container_client.list_blobs():
        blobs.append(i)
    latest_date = sorted([b['last_modified'] for b in blobs])[-1]
    time.sleep(1)
    # new container client
    container_client = blob_service_client.get_container_client(container_name)
    #container_client.list_blobs()
    #blobs = []
    #for i in container_client.list_blobs():
        #blobs.append(i)
    #print("Num blobs: {}".format(len(blobs)))
    file = open("status.txt","w+") 
    file.write("IDLE") 
    file.close()
    
    file = open("status_email.txt","w+") 
    file.write("--") 
    file.close()
    
    for blob in container_client.list_blobs():
        name = blob['name']
        t = blob['last_modified']
        if name not in image_history and t > latest_date:
            blob_client = blob_service_client.get_blob_client(container=container_name, blob=name)
            # save current image
            this_name = 'real_time/{}.jpg'.format(int(time.time()))
            with open(this_name, 'wb') as f:
                f.write(blob_client.download_blob().readall())
                
            time.sleep(1)
            if len(os.listdir('real_time'))!=0:
                last_img = sorted(os.listdir('real_time'))[-2]
                if last_img != '.DS_Store':
                    last_img = io.imread('real_time/{}'.format(last_img))
                    current_img_name = sorted(os.listdir('real_time'))[-1]
                    current_img = io.imread('real_time/{}'.format(current_img_name))
                    sim_idx = ssim(last_img, current_img, multichannel=True)
                    
                    print(sim_idx)
                    
                    if sim_idx <= 0.85:
                        
                        file = open("status.txt","w+") 
                        file.write("Inferencing") 
                        file.close() 
                        
                        shutil.rmtree("adian")
                        os.makedirs("adian")
                        print("New event found!!!!")
                        
                        fname = 'real_time/{}'.format(current_img_name)
                        obj_list = get_object_list(fname)
                        
                        file = open("status.txt","w+") 
                        file.write("Done!") 
                        file.close() 
                        
                        new_objs = [i for i in obj_list if i not in obj_list_history]
                        obj_list_history = obj_list_history + new_objs
                        check_recycle(new_objs, fname)
                        pickle.dump(obj_list_history, open("item_list.pkl", "wb" ))
                        
            image_history.append(name)

0.6516133242499812
New event found!!!!


tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

(1872, 1970, 3)
(1918, 1960, 3)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
mortar  :  89.18384909629822
mortar  :  97.2857654094696
Item Added.
Done.
Done.
0.6563862740904026
New event found!!!!
tracking <tf.Variable 'Variable_5:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_6:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_7:0' shape=(9, 4) dtype=float32> anchors
tracking <tf

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/envs/rectify/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-cb3ac6a7ae1b>", line 17, in <module>
    time.sleep(1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/rectify/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2034, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/envs/rectify/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1151, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/anaconda3/envs/rectify/lib/python3.7/site-pack

KeyboardInterrupt: 

In [12]:
obj_list_history

['mixing_bowl']

In [12]:
new_objs

[]